In [ ]:
%reload_ext autoreload
%autoreload 2

from configData import configVars
# videoNames = [
#     "New Quizzes Video",
#     "New Google Assignments in Canvas",
#     "Piazza Introduction Workshop",
#     "New Quizzes - Basics", # This transcript might fall back to K-Means in BERTopic for topic extraction.
#     "Sakai", # This transcript might fall back to K-Means in BERTopic for topic extraction.
#     "Rearrange Playlist video", # This transcript is too short for topic generation.
#     "IMSE 514 Presentation", # This transcript was retrieved from YouTube with atuomatic captions, and fails at sentence segmentation.
# ]

config = configVars()
# config.videoToUse = "Sakai"
config.setFromEnv()

In [ ]:
from transcriptLoader import retrieveTranscript
from questionGenerator import retrieveQuestions

videoData = retrieveTranscript(config)
questionData = retrieveQuestions(config, videoData=videoData)

# Additional code for Evaluation of Questions

The following code block is for generating an Excel file of the questions to be used for evaluating the generated questions. 
This is still not in a complete state, and is currently for protyping and testing only. 

You will need to install `tqdm`, `xlsxwriter` and possibly `openpyxl` Python packages via pip for the code to work.

In [ ]:
%reload_ext autoreload
%autoreload 2

fileList = [
    "IMSE 514 Presentation",
    "Kalpana-GenAI-Das",
    "Kalpana-maizey2-das",
    "Kalpana-U-M MaizeyDas",
    "New Google Assignments in Canvas",
    "New Quizzes - Basics",
    "New Quizzes Video",
    "Piazza Introduction Workshop",
    # "Rearrange Playlist video",
    "Sakai",
]

from configData import configVars, captionsFolder, outputFolder
from topicExtractor import retrieveTopics
from transcriptLoader import retrieveTranscript
from questionGenerator import retrieveQuestions
from questionGenerator import processCaptions
import pandas as pd
import glob
import os
import shutil
from tqdm import tqdm
from datetime import datetime
from pandas.io.formats import excel
excel.ExcelFormatter.header_style = None
wrappedCols  = {'Question': 'C', 'Answers':'G', 'Reason':'I'}

def makeExcelComparer(DFData, fileName, transcript):
    """
    Generate an Excel file with questions analysis based on the provided data.

    Args:
        DFData (pandas.DataFrame): DataFrame containing the data for analysis.
        fileName (str): Name of the file being analyzed.
        transcript (pandas.DataFrame): DataFrame containing the transcript data.

    Returns:
        None
    """
    dfSavePath = os.path.join(
            outputFolder,
            fileName,
            f"Questions Analysis - {fileName}.xlsx",
        )
    
    srtFile = glob.glob(
            os.path.join(captionsFolder, fileName, "*.srt")
        )[0]
    
    srtCopy = os.path.join(
        outputFolder,
        fileName,
        f"Transcript - {fileName}.srt",
    )
    
    shutil.copyfile(srtFile, srtCopy)

    mergedDF = pd.concat([DFData['LangChain'], DFData['BERTopic']]).reset_index(drop=False)

    for index, row in mergedDF.iterrows():
        start = datetime.strptime(row['Start'], '%H:%M:%S')
        end = datetime.strptime(row['End'], '%H:%M:%S')
        transcriptSlice = transcript[(transcript['Start'] >= start) & (transcript['End'] <= end)]
        if transcriptSlice.empty:
            transcriptSlice = transcript[(transcript['Start'] >= start)].head(1)
        relevantText = " ".join(transcriptSlice['Combined Lines'].tolist())

        mergedDF.at[index, 'Transcript'] = relevantText
    
    # display(mergedDF)

    with pd.ExcelWriter(dfSavePath, engine="xlsxwriter") as writer:
        mergedDF.to_excel(writer, sheet_name=fileName[:31])
        workbook = writer.book
        worksheet = writer.sheets[fileName[:31]]

        columns = list(mergedDF.columns)
        wrap_format = workbook.add_format({'text_wrap': True})
        header_format = workbook.add_format({'bold': True, 'text_wrap': True, 'align': 'center'})
        for col in columns:
            if col.startswith('Is this question:'):
                worksheet.set_column(columns.index(col)+1,columns.index(col)+1, 15, header_format)
            else:
                worksheet.set_column(columns.index(col)+1,columns.index(col)+1, None, wrap_format)

        for col in wrappedCols:
            excel_header  =  wrappedCols[col] + ':' + wrappedCols[col]
            worksheet.set_column(excel_header, 50, wrap_format)
        
        worksheet.set_column('M:M', 200, wrap_format)

        writer.close()


for fileName in tqdm(fileList):
    DFData = {}
    for generationModel in ['LangChain', 'BERTopic']:
    
        print(f"Processing {fileName} with {generationModel}...")
        config = configVars()
        config.setFromEnv()
        config.videoToUse = fileName
        config.generationModel = generationModel
        if config.generationModel == "LangChain":
            config.windowSize = 120

        videoData = retrieveTranscript(config)

        topicModeller = None
        if config.generationModel == "BERTopic":
            print(f"\t--> Retrieving Topics for {config.videoToUse}...")
            topicModeller = retrieveTopics(config, videoData)

        generatedData = retrieveQuestions(config, videoData=videoData, topicModeller=topicModeller)
        DFData[generationModel] = generatedData.makeDF()

    makeExcelComparer(DFData, fileName, videoData.combinedTranscript)